In [1]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split, TensorDataset, DataLoader, Subset
import random
import math
import matplotlib.pyplot as plt
import pandas as pd
import copy
import torchvision.transforms as T


import torch.nn.functional as F
from torchvision import datasets, transforms


In [2]:
repo_root = Path().resolve().parents[0]   # parent of "notebooks"
sys.path.insert(0, str(repo_root / "src"))

from fisher_information.fim import FisherInformationMatrix
from models.image_classification_models import *
from models.train_test import *
from prunning_methods.LTH import *

device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
repo_root

WindowsPath('D:/Mestrado/Fisher Information & Lottery Ticket/LTH-FisherInformation')

In [13]:
import os
os.chdir(repo_root)

output_dict = torch.load("saved_outputs/results/CONV-MNIST/LTH_mnist_convmodel.pth")

C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_9288\864958666.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  output_dict = torch.load("saved_outputs/results/CONV-MNIST/L

In [39]:
print('MNIST')
for i in range(100, 0, -10):
    fim = output_dict[i][0][1].fim['complete']
    fro = torch.norm(fim, p='fro').item()
    fro_diag = torch.norm(torch.diag(torch.diag(fim)), p='fro').item()
    fro_off_diag = torch.norm(fim - torch.diag(torch.diag(fim)), p='fro').item()
    fro_div = fro_off_diag / fro
    print(f"Percentage: {i}%, FIM Frobenius norm: {fro:.4f},  Off-diagonal FIM Frobenius norm: {fro_off_diag:.4f}, Ratio: {fro_div:.4f}, logdet_ratio: {output_dict[i][0][1].logdet_ratio:.2f}")

MNIST
Percentage: 100%, FIM Frobenius norm: 10.9937,  Off-diagonal FIM Frobenius norm: 10.9391, Ratio: 0.9950, logdet_ratio: 15294.70
Percentage: 90%, FIM Frobenius norm: 11.1720,  Off-diagonal FIM Frobenius norm: 11.1119, Ratio: 0.9946, logdet_ratio: 13178.50
Percentage: 80%, FIM Frobenius norm: 12.0044,  Off-diagonal FIM Frobenius norm: 11.9491, Ratio: 0.9954, logdet_ratio: 11829.61
Percentage: 70%, FIM Frobenius norm: 14.3312,  Off-diagonal FIM Frobenius norm: 14.2541, Ratio: 0.9946, logdet_ratio: 8399.47
Percentage: 60%, FIM Frobenius norm: 15.4669,  Off-diagonal FIM Frobenius norm: 15.3815, Ratio: 0.9945, logdet_ratio: 7159.66
Percentage: 50%, FIM Frobenius norm: 16.9487,  Off-diagonal FIM Frobenius norm: 16.8549, Ratio: 0.9945, logdet_ratio: 5416.67
Percentage: 40%, FIM Frobenius norm: 22.7995,  Off-diagonal FIM Frobenius norm: 22.6256, Ratio: 0.9924, logdet_ratio: 3575.42
Percentage: 30%, FIM Frobenius norm: 26.6644,  Off-diagonal FIM Frobenius norm: 26.4004, Ratio: 0.9901, logd

In [28]:
fim

tensor([[ 0.1705,  0.0680, -0.0812,  ..., -0.0119, -0.0131, -0.0068],
        [ 0.0680,  0.1619, -0.0576,  ..., -0.0073, -0.0099, -0.0004],
        [-0.0812, -0.0576,  0.2587,  ...,  0.0010,  0.0282,  0.0225],
        ...,
        [-0.0119, -0.0073,  0.0010,  ...,  0.0125, -0.0006, -0.0046],
        [-0.0131, -0.0099,  0.0282,  ..., -0.0006,  0.0182, -0.0021],
        [-0.0068, -0.0004,  0.0225,  ..., -0.0046, -0.0021,  0.0164]])

In [40]:
output_dict = torch.load("saved_outputs/results/CONV-EMNIST/LTH_emnist_convmodel.pth")

print('EMNIST')
for i in range(100, 0, -10):
    fim = output_dict[i][0][1].fim['complete']
    fro = torch.norm(fim, p='fro').item()
    fro_diag = torch.norm(torch.diag(torch.diag(fim)), p='fro').item()
    fro_off_diag = torch.norm(fim - torch.diag(torch.diag(fim)), p='fro').item()
    fro_div = fro_off_diag / fro
    print(f"Percentage: {i}%, FIM Frobenius norm: {fro:.4f},  Off-diagonal FIM Frobenius norm: {fro_off_diag:.4f}, Ratio: {fro_div:.4f}, logdet_ratio: {output_dict[i][0][1].logdet_ratio:.2f}")

C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_9288\1774352101.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  output_dict = torch.load("saved_outputs/results/CONV-EMNIST

EMNIST
Percentage: 100%, FIM Frobenius norm: 66.2708,  Off-diagonal FIM Frobenius norm: 65.2934, Ratio: 0.9853, logdet_ratio: 3777.31
Percentage: 90%, FIM Frobenius norm: 84.4447,  Off-diagonal FIM Frobenius norm: 83.0824, Ratio: 0.9839, logdet_ratio: 2941.37
Percentage: 80%, FIM Frobenius norm: 117.1330,  Off-diagonal FIM Frobenius norm: 114.6774, Ratio: 0.9790, logdet_ratio: 2852.86
Percentage: 70%, FIM Frobenius norm: 80.6000,  Off-diagonal FIM Frobenius norm: 78.7255, Ratio: 0.9767, logdet_ratio: 2452.16
Percentage: 60%, FIM Frobenius norm: 156.8210,  Off-diagonal FIM Frobenius norm: 152.8990, Ratio: 0.9750, logdet_ratio: 2423.85
Percentage: 50%, FIM Frobenius norm: 149.7093,  Off-diagonal FIM Frobenius norm: 144.7323, Ratio: 0.9668, logdet_ratio: 2109.68
Percentage: 40%, FIM Frobenius norm: 123.6767,  Off-diagonal FIM Frobenius norm: 119.3027, Ratio: 0.9646, logdet_ratio: 1700.50
Percentage: 30%, FIM Frobenius norm: 96.9264,  Off-diagonal FIM Frobenius norm: 92.4069, Ratio: 0.9534

In [41]:
output_dict = torch.load("saved_outputs/results/CONV-KMNIST/LTH_kmnist_convmodel.pth")

print('KMNIST')
for i in range(100, 0, -10):
    fim = output_dict[i][0][1].fim['complete']
    fro = torch.norm(fim, p='fro').item()
    fro_diag = torch.norm(torch.diag(torch.diag(fim)), p='fro').item()
    fro_off_diag = torch.norm(fim - torch.diag(torch.diag(fim)), p='fro').item()
    fro_div = fro_off_diag / fro
    print(f"Percentage: {i}%, FIM Frobenius norm: {fro:.4f},  Off-diagonal FIM Frobenius norm: {fro_off_diag:.4f}, Ratio: {fro_div:.4f}, logdet_ratio: {output_dict[i][0][1].logdet_ratio:.2f}")

C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_9288\3506409933.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  output_dict = torch.load("saved_outputs/results/CONV-KMNIST

KMNIST
Percentage: 100%, FIM Frobenius norm: 29.0177,  Off-diagonal FIM Frobenius norm: 28.8755, Ratio: 0.9951, logdet_ratio: 10505.37
Percentage: 90%, FIM Frobenius norm: 33.4418,  Off-diagonal FIM Frobenius norm: 33.2467, Ratio: 0.9942, logdet_ratio: 8419.55
Percentage: 80%, FIM Frobenius norm: 44.7345,  Off-diagonal FIM Frobenius norm: 44.4792, Ratio: 0.9943, logdet_ratio: 6340.33
Percentage: 70%, FIM Frobenius norm: 53.6446,  Off-diagonal FIM Frobenius norm: 53.2868, Ratio: 0.9933, logdet_ratio: 4832.82
Percentage: 60%, FIM Frobenius norm: 45.7420,  Off-diagonal FIM Frobenius norm: 45.4091, Ratio: 0.9927, logdet_ratio: 4569.75
Percentage: 50%, FIM Frobenius norm: 56.0381,  Off-diagonal FIM Frobenius norm: 55.5977, Ratio: 0.9921, logdet_ratio: 3504.48
Percentage: 40%, FIM Frobenius norm: 76.5632,  Off-diagonal FIM Frobenius norm: 75.9135, Ratio: 0.9915, logdet_ratio: 2650.10
Percentage: 30%, FIM Frobenius norm: 90.8324,  Off-diagonal FIM Frobenius norm: 89.7897, Ratio: 0.9885, logde

In [52]:
output_dict[100][0][2]

{'net.0.weight': tensor([[[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]],
 
 
         [[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]],
 
 
         [[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]],
 
 
         [[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]]], device='cuda:0'),
 'net.3.weight': tensor([[[[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]]],
 
 
         [[[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
  

In [71]:
aux = torch.argsort(torch.linalg.norm(output_dict[100][0][1].fim['complete'], dim=0))

In [94]:
mask_ravel = torch.concat([output_dict[90][0][2]['net.0.weight'].ravel(), torch.ones(output_dict[90][0][2]['net.0.weight'].shape[0], device='cuda'),output_dict[90][0][2]['net.3.weight'].ravel(), torch.ones(output_dict[90][0][2]['net.3.weight'].shape[0], device='cuda'),output_dict[90][0][2]['net.7.weight'].ravel(), torch.ones(output_dict[90][0][2]['net.7.weight'].shape[0], device='cuda')]).to('cpu')

In [98]:
aux [ torch.argwhere(mask_ravel == 0)]

tensor([[1520],
        [3920],
        [4810],
        [1125],
        [1610],
        [4012],
        [2109],
        [ 906],
        [ 398],
        [ 137],
        [3510],
        [3555],
        [2355],
        [1955],
        [ 847],
        [ 715],
        [  72],
        [ 913],
        [4514],
        [ 905],
        [1230],
        [4825],
        [  73],
        [ 331],
        [ 374],
        [1432],
        [4783],
        [4974],
        [1477],
        [3922],
        [ 727],
        [ 851],
        [2020],
        [  77],
        [1043],
        [1640],
        [1287],
        [4830],
        [4270],
        [ 625],
        [4903],
        [5070],
        [ 726],
        [ 414],
        [1925],
        [4929],
        [  75],
        [3120],
        [4546],
        [1182],
        [4892],
        [ 127],
        [2310],
        [ 295],
        [ 123],
        [1126],
        [1335],
        [1322],
        [1478],
        [  79],
        [3556],
        [1203],
        

In [99]:
a=1 